# What Does This Tool Do?

### Pulling from an API
    This tool pulls from an API hosted by USGS, a United States Government agency that tracks natural disasters. It specifically follows the coordinates (longitude and latitudinal) around the Middle East, a hub for earthquake activity. It then pulls all earthquakes above a magnitude of 3 from the past 365 days (1 year) in a 1500 km radius around Turkey. 

### Creating our Dataset
    Once we have called the API, we create a dataset that uses longitude and latitude received from the API to categorize different earthquakes. The full data on each earthquake include location, magnitude, time, latitude, longitude, and the weight (number of earthquakes in that location with an average in magnitude). 

### Understanding the Data
    We can then create a heatmap based on the location of each earthquake as well as create different tables that allow one to visualize the number of earthquakes, as well as compare different locations to determine high-risk areas. This allows for a very deep understanding of the level of danger to earthquakes different cities have. The hope for this project is to continue to use some AI model that will use previous data to better predict future occurrences. This dataset goes further back than 1 year, and I hope that I will be able to create better tables and graphs that depict the world of earthquakes in more depth.  


# Future of Earthquakes

### Location
    The Middle East is one of the most prominent locations of earthquakes in the modern era as they have torn apart many different communities. The data shown does not indicate the frequency of earthquakes increasing, but the scale of their devastation is. Dr. Brian Baptie, a seismologist with the British Geological Survey says that this is because the global population is increasing, with many living in densely populated areas (built-up). That includes areas such as Turkey in the Middle East and regions in India and Pakistan [2]. To note, the belt that we are specifically documenting with this tool is the Alpide earthquake belt, which travels through the Middle East and down towards the Indonesian region [2].


### Human Causes
    While contrary to popular perception, climate change is not a factor in earthquakes, human activities such as mining, reservoir-induced seismicity, or oil-and-gas extraction can contribute to seismic events in the Middle East according to Van Der Lee, a professor in Earth and Planetary Sciences at Northwestern University, US [2]. This phenomenon was documented in our class and can be seen throughout history among many different oil drilling operations. Another thing to note that specifically affects the Middle East is groundwater extraction. It has been observed to be linked to earthquakes in some parts of the Middle East – for instance, along the Dead Sea Fault in Jordan at the Wadi Al-Arab basin [2].


### AI to prevent Earthquakes
    Advancements in AI have revolutionized fields from medicine to finance, enabling unprecedented insights through data analysis and predictive modeling. Its capacity to process vast amounts of information has propelled human understanding by uncovering patterns and correlations previously unseen. This innovation has recently been turned towards natural disasters as an AI algorithm developed by researchers at The University of Texas at Austin correctly predicted 70% of earthquakes a week before they happened during a seven-month trial in China [1]. Specifically, the AI was trained to detect statistical bumps in real-time seismic data that had been paired with previous earthquakes. The outcome was a weekly forecast in which the AI successfully predicted 14 earthquakes within about 200 miles of where it estimated they would happen and at almost exactly the calculated strength. It missed one earthquake and gave eight false warnings [1]. This is a huge development as it can make many different methods of protection for people possible. Evacuations would be simplistic, as a week notice is surely enough time to evacuate even the most densely populated regions of the world. 



# Work Cited
    [1] “Ai-driven earthquake forecasting shows promise in trials,” PreventionWeb, https://www.preventionweb.net/news/ai-driven-earthquake-forecasting-shows-promise-trials (accessed Apr. 24, 2024). 
    
    [2] J. Bell, Seismic threats: Are earthquakes becoming more common ..., https://english.alarabiya.net/News/middle-east/2024/01/26/Are-earthquake-related-disasters-becoming-more-common-and-deadly-in-the-Middle-East- (accessed Apr. 24, 2024). 



# Github Link

    Attached below is the entire github link to my repository. There you can run the jupyter notebook yourself (given the prerequisites of Python and the required libraries). 
    
    https://github.com/QuinnB11/EARTH101-qdb5021

### Author: Quinn Butcher

In [49]:
import requests
import pandas as pd
import folium
from folium.plugins import HeatMap
import numpy as np
import branca.colormap as cm


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 500)

In [50]:
start_time = 'now-365days'
min_magnitude = 3
latitude = 39.1458
longitude = 34.1614
max_radius_km = 2500

In [51]:
url = requests.get(f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_time}&minmagnitude={min_magnitude}&latitude={latitude}&longitude={longitude}&maxradiuskm={max_radius_km}')
dataset = url.json()

In [52]:
features = dataset['features']

In [53]:
places = []
mags = []
times = []
lats = []
lons = []
weights = []

for feature in features:
    places.append(feature['properties']['place'])
    mags.append(feature['properties']['mag'])
    times.append(pd.to_datetime(feature['properties']['time'], unit='ms').strftime('%y/%m/%d %H:%M:%S'))
    lats.append(feature['geometry']['coordinates'][1])
    lons.append(feature['geometry']['coordinates'][0])
    weights.append(feature['properties']['mag'])

In [54]:
dataset_earthquake = pd.DataFrame({
                            'Place': places,
                            'Magnitude': mags,
                            'Time': times,
                            'Latitude': lats,
                            'Longitude': lons,
                            'Weight': weights
                            })

In [55]:
df = dataset_earthquake.copy()


In [56]:
df['Place'].fillna("NoInformation", inplace= True)

In [57]:
# Define coordinates for Turkey
middle_east_coord = [39, 35]

# Create a Folium map centered on Middle East
middle_east_map_normal = folium.Map(location=middle_east_coord, zoom_start=3.5)

# Create a heatmap layer based on the Latitude, Longitude, and Weight columns of DataFrame 'df'
HeatMap(data=df[['Latitude', 'Longitude', 'Weight']], radius=15).add_to(middle_east_map_normal)

# Iterate over each row in DataFrame 'df'
for index, row in df.iterrows():
    # Add a circle marker at each earthquake location with a radius based on the earthquake magnitude
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],  # Location of the circle marker
        radius=row['Magnitude']/100,  # Radius of the circle marker based on earthquake magnitude
        color='red',  # Border color of the circle marker
        fill_color='red'  # Fill color of the circle marker
    ).add_to(middle_east_map_normal)


# Display the map
middle_east_map_normal


In [58]:
# Iterate over unique places in the 'Place' column of DataFrame 'df'
for place in list(df['Place'].unique()):
    # Check if the place contains a comma and does not contain the word 'border'
    if "," in place and "border" not in place:
        # Check if 'Kahramanmaras' is in the place name, if so, set the country as 'Turkey', else extract the country from the place name
        if "Kahramanmaras" in place:
            df.loc[(df['Place'] == place), 'Country'] = 'Turkey'
        else:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(',')[-1])
    # Check if the place does not contain a comma, does not contain 'border', contains a space, has more than 1 and less than 3 words
    elif "," not in place and "border" not in place and " " in place and len(list(place.split(' ')))>1 and len(list(place.split(' ')))<3:
        # Check for specific countries or regions in the place name and assign the corresponding country
        if "Iran" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Iran"
        elif "Turkey" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Turkey"
        elif "Greece" in place:
            df.loc[(df['Place'] == place), 'Country'] = "Greece"
    # Check if the place does not contain a comma, does not contain 'border', and contains a space
    elif "," not in place and "border" not in place and " " in place :
        # Check for specific conditions in the place name and assign the country accordingly
        if "Kahramanmaras" in place:
             df.loc[(df['Place'] == place), 'Country'] = 'Turkey'
        elif "region" in place:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[0])
        elif "Sea" in place:
            df.loc[(df['Place'] == place), 'Country'] = place
        elif "coast" in place:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[-1])
        else:
            df.loc[(df['Place'] == place), 'Country'] = df['Place'].apply(lambda x : x.split(' ')[1])
    # Check if the place contains the word 'border', assign the place as the country
    elif "border" in place:
        df.loc[(df['Place'] == place), 'Country'] = place
    else:
        df.loc[(df['Place'] == place), 'Country'] = place

# Fill missing country information for specific conditions
df.loc[((df['Country'].isnull()) & (df['Place'].str.contains('Cyprus'))), 'Country' ] = 'Cyprus'
df.loc[((df['Country'].isnull()) & (df['Place'].str.contains('Sea'))), 'Country' ] = df['Place']        

# Extract city information from places containing a comma
for place in list(df['Place'].unique()):
    if "," in place:
        df.loc[(df['Place'] == place), 'City'] = df['Place'].apply(lambda x: x.split(',')[0].split(' ')[-1])

# Fill missing city information with a default value
df['City'].fillna('NO CITY INFORMATION' ,inplace = True)

df['Date'] = df['Time'].apply(lambda x: x.split(" ")[0])
df['TimeZone'] = df['Time'].apply(lambda x: x.split(" ")[1])

df['Year\Month'] = df['Date'].apply(lambda x : str(x.split("/")[0]) + "/" +  str(x.split("/")[1]))


## How many earthquakes have occurred in which country in the last 365 Days?

In [59]:
df.groupby('Country').agg({"Weight": "count"}).sort_values(by = "Weight", ascending = False)[:10]

,Weight
Country,
Greece,134
Turkey,132
Iran,120
Italy,31
Turkey,14
Romania,13
Russia,9
Turkmenistan,8
Bosnia and Herzegovina,8


## What is the average magnitude of these earthquakes based on city and country? 

In [60]:
table_ea = df[df['Country'] != "NoInformation"].groupby(['Year\Month','Country', 'City']).agg({"Magnitude": "count",
                                                      "Weight" : "mean"}).sort_values(by = "Magnitude", ascending = False)

table_ea.columns = ['Count', 'Magnitude']
table_ea

,,,Count,Magnitude
Year\Month,Country,City,,
23/05,Turkey,Saimbeyli,6,4.383
24/04,Turkey,Sulusaray,5,4.580
23/10,Iran,Abbas,5,5.060
23/05,Turkey,NO CITY INFORMATION,5,4.220
23/08,Hungary,Szarvas,4,3.550
23/05,Greece,Moíres,4,4.675
24/02,Greece,Palaióchora,4,4.375
24/03,Iran,Gerāsh,4,4.175
23/07,Turkey,NO CITY INFORMATION,3,3.967
